In [1]:
import torch
import torchvision

In [2]:
import arrayfire as af
import array

In [3]:
def toArrayFire(x):
    x_np = x.detach().contiguous().numpy()
    shape = 1
    if len(x_np.shape) == 0:
        shape = (1,)
    else:
        shape = x_np.shape[::-1]
    print(shape)
    afArray = af.Array(x_np.ctypes.data, shape, x_np.dtype.char)
    return afArray

def saveStateDict(module, filepath):
    i = 0
    for (name, param) in module.named_parameters():
        #param = module.state_dict()[name]
        print(name, "\t", param.size())
        if 'in_proj' in name:
            print(param.shape)
            q, k, v = param.chunk(3, dim=0)
            print('in_proj!')
            af.array.save_array(name + 'q', toArrayFire(q), filepath, True)
            af.array.save_array(name + 'k', toArrayFire(k), filepath, True)
            af.array.save_array(name + 'v', toArrayFire(k), filepath, True)
            continue
        if len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            af.array.save_array(name, af_array, filepath, True)
            i = i + 1
    print(i)
    for name in module.state_dict():
        if 'running' in name:
            print(name)
            af_array = toArrayFire(module.state_dict()[name])
            af.array.save_array(name, af_array, filepath + 'running', True)

In [4]:
filepath = '/private/home/padentomasello/scratch/pytorch_testing/basic_linear.array'
batchSize = 2
inChannels = 5
outChannels = 7

x = torch.rand(batchSize, inChannels)
x_af = toArrayFire(x)

af.array.save_array('input', x_af, filepath)
model = torch.nn.Linear(inChannels, outChannels, False)
weight = torch.stack([torch.arange(0, inChannels, dtype=torch.float32) for i in range(outChannels)], 0)
print(weight.shape)
model.weight = torch.nn.Parameter(weight)
output = model(x)
saveStateDict(model, filepath)
af.array.save_array('output', toArrayFire(output), filepath, True)

(5, 2)
torch.Size([7, 5])
weight 	 torch.Size([7, 5])
(5, 7)
1
(7, 2)


2

In [5]:
output

tensor([[7.2556, 7.2556, 7.2556, 7.2556, 7.2556, 7.2556, 7.2556],
        [3.5327, 3.5327, 3.5327, 3.5327, 3.5327, 3.5327, 3.5327]],
       grad_fn=<MmBackward>)

In [19]:
from models.detr import *
from models.matcher import *
class NestedTensor(object):

    def __init__(self, mask):
        self.mask = mask
        self.tensors = torch.ones(1)

filepath = '/private/home/padentomasello/scratch/pytorch_testing/matcher.array'

matcher = HungarianMatcher(1, 1, 1)
batch_size = 2
num_queries = 20
num_classes = 91

pred_logits = torch.rand(batch_size, num_queries, num_classes + 1, requires_grad=True)
pred_boxes = torch.rand(batch_size, num_queries, 4, requires_grad=True)
outputs = {
    "pred_logits":pred_logits,
    "pred_boxes":pred_boxes
}

targets = [
{'boxes': torch.tensor([[0.4395, 0.3252, 0.0628, 0.1583],
        [0.4474, 0.3120, 0.3526, 0.4358],
        [0.5994, 0.0650, 0.2941, 0.1118]]),
 'labels': torch.tensor([32,  1, 85]),
},
 {'boxes': torch.tensor([[0.6240, 0.2875, 0.1698, 0.3400],
        [0.2351, 0.5314, 0.4701, 0.9371],
        [0.4722, 0.6277, 0.2816, 0.7445],
        [0.7453, 0.5528, 0.5094, 0.8945],
        [0.6128, 0.6995, 0.1498, 0.0647],
        [0.4731, 0.4819, 0.1561, 0.0233],
        [0.9721, 0.2031, 0.0187, 0.1447],
        [0.9882, 0.1968, 0.0236, 0.1408],
        [0.9960, 0.6344, 0.0080, 0.4890],
        [0.1050, 0.2647, 0.2100, 0.4400],
        [0.9948, 0.4351, 0.0104, 0.0984],
        [0.9401, 0.1980, 0.0253, 0.1481]]), 
  'labels': torch.tensor([ 1,  1,  1,  1, 51, 48, 50, 50, 79,  1, 51, 50])
 }
]
#targets = [
#{'boxes': torch.tensor([[0.4395, 0.3252, 0.0628, 0.1583],
#        [0.4474, 0.3120, 0.3526, 0.4358],
#        [0.5994, 0.0650, 0.2941, 0.1118]]),
# 'labels': torch.tensor([0,  1, 1]),}
#]

base_filepath = '/private/home/padentomasello/scratch/pytorch_testing/matcher_test0'
bbox_filepath = base_filepath + '_bboxes.array'
label_filepath = base_filepath + '_labels.array'
af.array.save_array('pred_logits', toArrayFire(pred_logits), base_filepath + 'input.array')
af.array.save_array('pred_boxes', toArrayFire(pred_boxes), base_filepath + 'input.array', True)
bboxes = [ toArrayFire(target["boxes"]) for target in targets ]
labels = [ toArrayFire(target["labels"]) for target in targets ]
for box in bboxes:
    af.array.save_array('boxes', box, bbox_filepath, True)
    
for label in labels:
    af.array.save_array('label', label, label_filepath, True)


result = matcher.forward(outputs, targets)

(92, 20, 2)
(4, 20, 2)
(4, 3)
(4, 12)
(3,)
(12,)


In [23]:
from models.detr import *
base_filepath = '/private/home/padentomasello/scratch/pytorch_testing/set_criterion'
bbox_filepath = base_filepath + '_bboxes.array'
label_filepath = base_filepath + '_labels.array'
loss_filepath = base_filepath + '_loss.array'
grad_filepath = base_filepath + '_grad.array'
pred_logits.grad = None
pred_boxes.grad = None
print(af.array.save_array('pred_logits', toArrayFire(pred_logits), base_filepath + '_input.array', False))
print(af.array.save_array('pred_boxes', toArrayFire(pred_boxes), base_filepath + '_input.array', True))
bboxes = [ toArrayFire(target["boxes"]) for target in targets ]
labels = [ toArrayFire(target["labels"]) for target in targets ]
append = False
for box in bboxes:
    print('here2', af.array.save_array('boxes', box, bbox_filepath, append))
    append = True
append = False    
for label in labels:
    print('here', af.array.save_array('label', label, label_filepath, append))
    append = True


result = matcher.forward(outputs, targets)
weight_dict = {'loss_ce': 1, 'loss_bbox': 1}
weight_dict['loss_giou'] = 1
losses = []
losses.extend(['boxes', 'cardinality'])
losses.append('labels')
eos_coef = 0.1
crit = SetCriterion(num_classes, matcher, weight_dict, eos_coef, losses)
append = False
losses = crit.forward(outputs, targets)
for key, array in losses.items():
    print(key, af.array.save_array(key + '_0', toArrayFire(array), loss_filepath, append))
    #array.backward(retain_graph=True)
    append = True
losses['loss_bbox'].backward(retain_graph=True)
losses['loss_giou'].backward(retain_graph=True)
losses['loss_ce'].backward()


(92, 20, 2)
0
(4, 20, 2)
1
(4, 3)
(4, 12)
(3,)
(12,)
here2 0
here2 1
here 0
here 1
[(tensor([0, 6, 7]), tensor([2, 1, 0])), (tensor([ 0,  2,  5,  6,  8, 11, 12, 14, 15, 16, 17, 19]), tensor([10,  2,  5,  7,  8,  3,  1, 11,  6,  0,  4,  9]))]
here
tensor([[85, 91, 91, 91, 91, 91,  1, 32, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91,
         91, 91],
        [51, 91,  1, 91, 91, 48, 50, 91, 79, 91, 91,  1,  1, 91, 50, 50,  1, 51,
         91,  1]])
tensor([[[0.8771, 0.3445, 0.7739,  ..., 0.1834, 0.6261, 0.7407],
         [0.8994, 0.7917, 0.7706,  ..., 0.2017, 0.0570, 0.4281],
         [0.6708, 0.4379, 0.3421,  ..., 0.4119, 0.7743, 0.7403],
         ...,
         [0.6878, 0.7168, 0.9990,  ..., 0.7257, 0.3486, 0.8485],
         [0.2410, 0.3199, 0.8637,  ..., 0.7191, 0.5703, 0.7800],
         [0.6612, 0.8621, 0.5733,  ..., 0.4579, 0.3235, 0.5163]],

        [[0.3773, 0.9716, 0.1230,  ..., 0.0232, 0.1671, 0.3842],
         [0.1942, 0.9531, 0.5105,  ..., 0.5641, 0.0799, 0.9404],
         [0.7644, 

In [25]:
print(outputs["pred_boxes"].grad)
print(outputs["pred_logits"].grad)
print(af.array.save_array('pred_boxes', toArrayFire(outputs["pred_boxes"].grad), grad_filepath, False))
print(af.array.save_array('pred_logits', toArrayFire(outputs["pred_logits"].grad), grad_filepath, True))

tensor([[[-0.2190, -0.0667,  0.0057,  0.1792],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.1743,  0.1556,  0.0147,  0.0534],
         [ 0.1074, -0.0667,  0.0208, -0.1948],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.0667, -0.0667,  0.0668,  0.0668],
         [ 

In [9]:
loss_filepath

'/private/home/padentomasello/scratch/pytorch_testing/set_criterion_loss.array'

In [10]:
filepath = '/private/home/padentomasello/scratch/pytorch_testing/weight_cross_entropy.array'

num_classes = 2
weights = torch.ones(num_classes)
weights[0:1] = 0.1
input = torch.randn(3, num_classes, requires_grad=True)
target = torch.randint(num_classes, (3,), dtype=torch.int64)
af.array.save_array('input', toArrayFire(input), filepath, False)
af.array.save_array('weight', toArrayFire(weights), filepath, True)
af.array.save_array('target', toArrayFire(target), filepath, True)
F.cross_entropy(input, target, weights)

(2, 3)
(2,)
(3,)


tensor(1.7808, grad_fn=<NllLossBackward>)

In [11]:
softmaxed = torch.nn.functional.log_softmax(input, 1)
loss = torch.nn.functional.nll_loss(softmaxed, target, weights, None, -1, None, 'mean')

In [12]:
loss.backward()

In [13]:
input.grad

tensor([[-0.0511,  0.0511],
        [ 0.7241, -0.7241],
        [-0.0077,  0.0077]])

In [14]:
softmaxed

tensor([[-0.9484, -0.4900],
        [-0.1404, -2.0324],
        [-0.0971, -2.3803]], grad_fn=<LogSoftmaxBackward>)

In [15]:
torch.nn.functional.nll_loss(softmaxed, target, weights, None, -1, None, 'none')

tensor([0.0948, 2.0324, 0.0097], grad_fn=<NllLossBackward>)

In [16]:
torch.nn.functional.nll_loss(softmaxed, target, weights, None, -1, None, 'mean')

tensor(1.7808, grad_fn=<NllLossBackward>)

In [17]:
torch.nn.functional.nll_loss(softmaxed, target, weights, None, -1, None, 'sum')

tensor(2.1369, grad_fn=<NllLossBackward>)

In [18]:
target

tensor([0, 1, 0])

In [19]:
torch.nn.functional.nll_loss(softmaxed, target, None, None, -1, None, 'none')

tensor([0.9484, 2.0324, 0.0971], grad_fn=<NllLossBackward>)

In [20]:
torch.nn.functional.nll_loss(softmaxed, target, None, None, -1, None, 'sum')

tensor(3.0779, grad_fn=<NllLossBackward>)

In [21]:
torch.nn.functional.nll_loss(softmaxed, target, None, None, -1, None, 'mean')

tensor(1.0260, grad_fn=<NllLossBackward>)